### PLAN - FEATURE ENGINEERING   

#### 1. TRAITEMENT/FUSION DES DATASETS
  
TRAIN + TEST :  
- sauvegarde IDs + fusion  
- traitement code_gender  
- traitement days_employed  
- traitement variables catégorielles (binaires et non binaires)  
- création features (5)  
  
BUREAU + BUREAU_BAL :  
- traitement variables catégorielles non binaires  
- aggrégations bureau_bal et fusion avec bureau  
- suppression sk_id_bureau  
- aggrégations bureau et regroupement par sk_id_curr  
- création feature (1)  
- fusion avec df_global  
  
PREVIOUS_APPLICATION :  
- suppression outliers days 
- aggrégations
- création feature (1)  
- traitement variables catégorielles  
- fusion avec df_global  
  
POS_CASH_BALANCE :  
- traitement variables catégorielles 
- aggrégations  
- création feature (1)  
- fusion avec df_global  
  
INSTALLMENTS PAYMENTS :  
- traitement variables catégorielles  
- aggrégations  
- création features (5)  
- fusion avec df_global
  
CREDIT CARD BALANCE :  
- traitement variables catégorielles  
- aggrégations  
- création feature (1)  
- fusion avec df_global

#### 2. ETUDE DES CORRELATIONS

#### 3. VALEURS NULLES   
  
#### 4. VALEURS INFINIES 
  
#### 5. EXPORT DATASETS TRAIN ET TEST

  
  
  
#### Création features :  
  
- 'DAYS_EMPLOYED_PERCENT'  
- 'INCOME_CREDIT_PERCENT' 
- 'INCOME_PER_PERSON'  
- 'ANNUITY_INCOME_PERCENT'  
- 'PAYMENT_RATE'  
    
    
- 'BUREAU_COUNT' 
    
    
- 'APP_CREDIT_PERCENT'
    
    
- 'POS_COUNT'  
    
    
- 'PAYMENT_PERCENT'  
- 'PAYMENT_DIFF'  
- 'DAYS_PAST_DUE'  
- 'DAYS_BEFORE_DUE'  
- 'INSTAL_COUNT'  
    
    
- 'CC_COUNT'

## Packages

In [13]:
import os
import re
import glob
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from zipfile import ZipFile

from sklearn.preprocessing import LabelEncoder

## Fonctions

In [14]:
# Fonction d'utilisation de OneHotEncoder
def one_hot_encoding(data, nan_as_category=True):

    original_cols = data.columns.to_list()
    categorical_cols = data.select_dtypes(object).columns.to_list()

    data = pd.get_dummies(data, columns=categorical_cols, dummy_na=nan_as_category)
    new_columns = [col for col in data.columns if col not in original_cols]

    return data, new_columns

In [15]:
# Fonction de renommage
def rename_grouped_index(grouped_df, prefix=''):
    
    return pd.Index([prefix + elem[0] + "_" + elem[1].upper() for elem in grouped_df.columns.tolist()])

In [16]:
# Fonction relative aux valeurs nulles
def missing_values(data, plot=False):
    
    mst = pd.DataFrame(
        {"Num_Missing": data.isnull().sum(), "Missing_Ratio": data.isnull().sum() / data.shape[0]}).sort_values(
        "Num_Missing", ascending=False)
    mst["DataTypes"] = data[mst.index].dtypes.values
    mst = mst[mst.Num_Missing > 0].reset_index().rename({"index": "Feature"}, axis=1)

    print("Nb de features comprenant des valeurs nulles : ", mst.shape[0], "\n")

    if mst[mst.Missing_Ratio >= 0.3].shape[0] > 0:
        print("Features ne contenant que des valeurs nulles :", mst[mst.Missing_Ratio >= 1.0].Feature.tolist())
        data.drop(mst[mst.Missing_Ratio >= 1.0].Feature.tolist(), axis=1, inplace=True)

        print("Suppresion des features entièrement vides : OK", "\n")
    if plot:
        plt.figure(figsize=(25, 8))
        p = sns.barplot(mst.Feature, mst.Missing_Ratio)
        for rotate in p.get_xticklabels():
            rotate.set_rotation(90)
        plt.show()

    print(mst, "\n")

## Chemins

In [17]:
# Chemin du dataset initial (external files)
path = './external_files/'
folder_path = glob.glob('./external_files/*.csv')

# Liste des fichiers du dataset (total : 10), et nommage
train = pd.read_csv(path + 'application_train.csv')
train.name = 'train'

test = pd.read_csv(path + 'application_test.csv')
test.name = 'test'

bureau = pd.read_csv(path + 'bureau.csv')
bureau.name = 'bureau'

bureau_bal = pd.read_csv(path + 'bureau_balance.csv')
bureau_bal.name = 'bureau_bal'

pos_cash_bal = pd.read_csv(path + 'POS_CASH_balance.csv')
pos_cash_bal.name = 'pos_cash_bal'

cred_card_bal = pd.read_csv(path + 'credit_card_balance.csv')
cred_card_bal.name = 'cred_card_bal'

prev_app = pd.read_csv(path + 'previous_application.csv')
prev_app.name = 'prev_app'

install_pay = pd.read_csv(path + 'installments_payments.csv')
install_pay.name = 'install_pay'

col_descr = pd.read_csv(path + 'HomeCredit_columns_description.csv', encoding_errors='ignore')
col_descr.name = 'col_descr'

sample_sub = pd.read_csv(path + 'sample_submission.csv', encoding_errors='ignore')
sample_sub.name = 'sample_sub'

## 1. Traitement / fusion des datasets

#### APPLICATION_TRAIN + APPLICATION_TEST

In [18]:
# Copie des fichiers pour abalyse
df_train = train.copy()
df_test = test.copy()
split_rate = round(1 - (df_train.shape[0] / (df_train.shape[0] + df_test.shape[0])), 3)

# Affichage formats des fichiers
print('Format df_train :')
display(df_train.shape)
print('Format df_test :')
display(df_test.shape)
print('Split train/test : {}%'.format(split_rate))
print('\nFormat cumulé : {} x {}'.format((df_train.shape[0] + df_test.shape[0]), df_train.shape[1]))
print('\nComposition des features du dataset train : \n{}'.format(df_train.dtypes.value_counts()))

Format df_train :


(307511, 122)

Format df_test :


(48744, 121)

Split train/test : 0.137%

Format cumulé : 356255 x 122

Composition des features du dataset train : 
float64    65
int64      41
object     16
Name: count, dtype: int64


#### Fusion des dataset pour application du features engineering

In [19]:
# Sauvegarde des IDs avant fusion (pour split ultérieur)
test_ids = df_test['SK_ID_CURR'].to_list()

# Fusion des dataset
df_global = pd.concat([df_train, df_test])

# Affichage
df_global

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,NaN,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,NaN,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,NaN,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,NaN,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


Rappel des éléments identifiés lors de l'EDA :  
  
  - Traitement outliers 'DAYS_EMPLOYED'  
  - Suppression CODE_GENDER = 'XNA'  
  - Création de features

#### Traitement des outliers

In [20]:
# Traitement des CODE_GENDER = XNA
xna_list = df_global[df_global['CODE_GENDER'] == 'XNA'].index
df_global = df_global.drop(xna_list, axis = 0)
print('Suppression de {} lignes'.format(len(xna_list)))
print('Nouveau format : {}'.format(df_global.shape))

Suppression de 4 lignes
Nouveau format : (356249, 122)


In [21]:
# Remplacement des outliers 'DAYS_EMPLOYED' par NaN
days_employed_outliers = df_global['DAYS_EMPLOYED'].max()
df_global = df_global.replace({days_employed_outliers : np.nan})
print("Remplacement de l'outliers {} par NaN".format(days_employed_outliers))

Remplacement de l'outliers 365243 par NaN


#### Création de features

In [22]:
# % d'emploi depuis la naissance
df_global['DAYS_EMPLOYED_PERCENT'] = df_global['DAYS_EMPLOYED'] / df_global['DAYS_BIRTH']

# % Taux d'endettement
df_global['INCOME_CREDIT_PERCENT'] = df_global['AMT_INCOME_TOTAL'] / df_global['AMT_CREDIT']

# Revenu moyen par pers. du foyer
df_global['INCOME_PER_PERSON'] = df_global['AMT_INCOME_TOTAL'] / df_global['CNT_FAM_MEMBERS']

# Part des annuités de crédit sur les revenus totaux
df_global['ANNUITY_INCOME_PERCENT'] = df_global['AMT_ANNUITY'] / df_global['AMT_INCOME_TOTAL']

# Part annuelle du remboursement du crédit
df_global['PAYMENT_RATE'] = df_global['AMT_ANNUITY'] / df_global['AMT_CREDIT']

print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 127)


#### Traitement des variables catégorielles

Factorisation simple pour les variables catégorielles binaires, et utilisation de OneHotEncoder pour les autres.

In [23]:
# Variables catégorielles binaires (factorisation)
binary_feat = [col for col in df_global.select_dtypes(object).columns if df_global[col].nunique() == 2]

for col in binary_feat:
    df_global[col], uniques = pd.factorize(df_global[col])
display(df_global[binary_feat].head())

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,EMERGENCYSTATE_MODE
0,0,0,0,0,0
1,0,1,0,1,0
2,1,0,1,0,-1
3,0,1,0,0,-1
4,0,0,0,0,-1


In [24]:
# Variables catégorielles non binaires (OneHotEncoder)
df_global, train_encoded_cols = one_hot_encoding(df_global, nan_as_category=False)
display(df_global.head())
display(df_global.shape)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden
0,100002.0,1.0,0,0,0,0,0,202500.0,406597.5,24700.5,...,True,False,False,False,False,False,False,False,True,False
1,100003.0,0.0,0,1,0,1,0,270000.0,1293502.5,35698.5,...,True,False,False,True,False,False,False,False,False,False
2,100004.0,0.0,1,0,1,0,0,67500.0,135000.0,6750.0,...,False,False,False,False,False,False,False,False,False,False
3,100006.0,0.0,0,1,0,0,0,135000.0,312682.5,29686.5,...,False,False,False,False,False,False,False,False,False,False
4,100007.0,0.0,0,0,0,0,0,121500.0,513000.0,21865.5,...,False,False,False,False,False,False,False,False,False,False


(356249, 245)

#### Fichiers : BUREAU + BUREAU_BALANCE

In [25]:
# Variables catégorielles binaires ?
binary_feat_bureau = [col for col in bureau.select_dtypes(object).columns if bureau[col].nunique() == 2]
binary_feat_bureau_bal = [col for col in bureau_bal.select_dtypes(object).columns if bureau_bal[col].nunique() == 2]
print('Nombre de variables catégorielles binaires BUREAU : {}'.format(len(binary_feat_bureau)))
print('Nombre de variables catégorielles binaires BUREAU_BAL : {}'.format(len(binary_feat_bureau_bal)))

Nombre de variables catégorielles binaires BUREAU : 0
Nombre de variables catégorielles binaires BUREAU_BAL : 0


In [26]:
# Traitement des variables catégorielles non binaires (OneHotEncoder)
bureau, bureau_encoded_cols = one_hot_encoding(bureau, nan_as_category = True)
bureau_bal, bureau_bal_encoded_cols = one_hot_encoding(bureau_bal, nan_as_category = True)

display(bureau)
display(bureau_bal)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,...,False,False,False,False,False,False,False,False,False,False
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,...,False,False,False,False,False,False,False,False,False,False
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.50,NaN,...,False,False,False,False,False,False,False,False,False,False
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.00,NaN,...,False,False,False,False,False,False,False,False,False,False
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,...,False,False,False,False,True,False,False,False,False,False
1716424,100044,5057754,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,...,False,False,False,False,False,False,False,False,False,False
1716425,100044,5057762,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,...,False,False,False,False,False,False,False,False,False,False
1716426,246829,5057770,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,...,False,False,False,False,False,False,False,False,False,False


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,STATUS_nan
0,5715448,0,False,False,False,False,False,False,True,False,False
1,5715448,-1,False,False,False,False,False,False,True,False,False
2,5715448,-2,False,False,False,False,False,False,True,False,False
3,5715448,-3,False,False,False,False,False,False,True,False,False
4,5715448,-4,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
27299920,5041336,-47,False,False,False,False,False,False,False,True,False
27299921,5041336,-48,False,False,False,False,False,False,False,True,False
27299922,5041336,-49,False,False,False,False,False,False,False,True,False
27299923,5041336,-50,False,False,False,False,False,False,False,True,False


In [27]:
# Définition des aggrégations de bureau_bal
bureau_bal_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}

for col in bureau_bal_encoded_cols:
    bureau_bal_aggregations[col] = ['mean']
print(bureau_bal_aggregations)

{'MONTHS_BALANCE': ['min', 'max', 'size'], 'STATUS_0': ['mean'], 'STATUS_1': ['mean'], 'STATUS_2': ['mean'], 'STATUS_3': ['mean'], 'STATUS_4': ['mean'], 'STATUS_5': ['mean'], 'STATUS_C': ['mean'], 'STATUS_X': ['mean'], 'STATUS_nan': ['mean']}


In [28]:
# Regroupement par 'SK_ID_BUREAU'
bureau_bal_grouped = bureau_bal.groupby('SK_ID_BUREAU').agg(bureau_bal_aggregations)

# Renommage des colonnes
bureau_bal_grouped.columns = rename_grouped_index(bureau_bal_grouped)

# Fusion avec bureau
bureau = bureau.join(bureau_bal_grouped, how = 'left', on = 'SK_ID_BUREAU')

# Suppression de 'SK_ID_BUREAU'
bureau.drop(['SK_ID_BUREAU'], axis = 1, inplace = True)
bureau

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,MONTHS_BALANCE_SIZE,STATUS_0_MEAN,STATUS_1_MEAN,STATUS_2_MEAN,STATUS_3_MEAN,STATUS_4_MEAN,STATUS_5_MEAN,STATUS_C_MEAN,STATUS_X_MEAN,STATUS_nan_MEAN
0,215354,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716424,100044,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716425,100044,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1716426,246829,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Définition des aggrégations de bureau fusionné avec bureau_bal
bureau_aggregations = {'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
                       'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
                       'DAYS_CREDIT_UPDATE': ['mean'],
                       'CREDIT_DAY_OVERDUE': ['max', 'mean'],
                       'AMT_CREDIT_MAX_OVERDUE': ['mean'],
                       'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
                       'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
                       'AMT_CREDIT_SUM_OVERDUE': ['mean'],
                       'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
                       'AMT_ANNUITY': ['max', 'mean'],
                       'CNT_CREDIT_PROLONG': ['sum'],
                       'MONTHS_BALANCE_MIN': ['min'],
                       'MONTHS_BALANCE_MAX': ['max'],
                       'MONTHS_BALANCE_SIZE': ['mean', 'sum']}

for col in bureau_encoded_cols:
    bureau_aggregations[col] = ['mean']
for col in bureau_bal_encoded_cols:
    bureau_aggregations[col + "_MEAN"] = ['mean']
    
print(bureau_aggregations)

{'DAYS_CREDIT': ['min', 'max', 'mean', 'var'], 'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'], 'DAYS_CREDIT_UPDATE': ['mean'], 'CREDIT_DAY_OVERDUE': ['max', 'mean'], 'AMT_CREDIT_MAX_OVERDUE': ['mean'], 'AMT_CREDIT_SUM': ['max', 'mean', 'sum'], 'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'], 'AMT_CREDIT_SUM_OVERDUE': ['mean'], 'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'], 'AMT_ANNUITY': ['max', 'mean'], 'CNT_CREDIT_PROLONG': ['sum'], 'MONTHS_BALANCE_MIN': ['min'], 'MONTHS_BALANCE_MAX': ['max'], 'MONTHS_BALANCE_SIZE': ['mean', 'sum'], 'CREDIT_ACTIVE_Active': ['mean'], 'CREDIT_ACTIVE_Bad debt': ['mean'], 'CREDIT_ACTIVE_Closed': ['mean'], 'CREDIT_ACTIVE_Sold': ['mean'], 'CREDIT_ACTIVE_nan': ['mean'], 'CREDIT_CURRENCY_currency 1': ['mean'], 'CREDIT_CURRENCY_currency 2': ['mean'], 'CREDIT_CURRENCY_currency 3': ['mean'], 'CREDIT_CURRENCY_currency 4': ['mean'], 'CREDIT_CURRENCY_nan': ['mean'], 'CREDIT_TYPE_Another type of loan': ['mean'], 'CREDIT_TYPE_Car loan': ['mean'], 'CREDIT_TYPE_Cash loan (non-

In [30]:
# Regroupement par ID client ('SK_ID_CURR')
bureau_grouped = bureau.groupby('SK_ID_CURR').agg(bureau_aggregations)

# Renommage des colonnes
bureau_grouped.columns = rename_grouped_index(bureau_grouped, prefix='BUREAU_')

bureau_grouped

,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_VAR,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_DAYS_CREDIT_UPDATE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_CREDIT_DAY_OVERDUE_MEAN,...,BUREAU_CREDIT_TYPE_nan_MEAN,BUREAU_STATUS_0_MEAN_MEAN,BUREAU_STATUS_1_MEAN_MEAN,BUREAU_STATUS_2_MEAN_MEAN,BUREAU_STATUS_3_MEAN_MEAN,BUREAU_STATUS_4_MEAN_MEAN,BUREAU_STATUS_5_MEAN_MEAN,BUREAU_STATUS_C_MEAN_MEAN,BUREAU_STATUS_X_MEAN_MEAN,BUREAU_STATUS_nan_MEAN_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,240043.666667,-1329.0,1778.0,82.428571,-93.142857,0,0.0,...,0.0,0.336651,0.007519,0.0,0.0,0.0,0.0,0.441240,0.214590,0.0
100002,-1437,-103,-874.000000,186150.000000,-1072.0,780.0,-349.000000,-499.875000,0,0.0,...,0.0,0.406960,0.255682,0.0,0.0,0.0,0.0,0.175426,0.161932,0.0
100003,-2586,-606,-1400.750000,827783.583333,-2434.0,1216.0,-544.500000,-816.000000,0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-1326,-408,-867.000000,421362.000000,-595.0,-382.0,-488.500000,-532.000000,0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-373,-62,-190.666667,26340.333333,-128.0,1324.0,439.333333,-54.333333,0,0.0,...,0.0,0.735043,0.000000,0.0,0.0,0.0,0.0,0.128205,0.136752,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-2713,-483,-1667.076923,407302.243590,-2499.0,1363.0,-1232.333333,-1064.538462,0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
456250,-1002,-760,-862.000000,15724.000000,-272.0,2340.0,1288.333333,-60.333333,0,0.0,...,0.0,0.130259,0.000000,0.0,0.0,0.0,0.0,0.252525,0.617216,0.0
456253,-919,-713,-867.500000,10609.000000,-189.0,1113.0,280.500000,-253.250000,0,0.0,...,0.0,0.404906,0.000000,0.0,0.0,0.0,0.0,0.459677,0.135417,0.0


In [31]:
# Création feature : nombre de prêts par client
bureau_grouped['BUREAU_COUNT'] = bureau.groupby('SK_ID_CURR').size()

In [32]:
# Fusion de bureau_grouped (bureau + bureau_bal) avec df_global
df_global = df_global.join(bureau_grouped, how = 'left', on = 'SK_ID_CURR')

print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 308)


#### Fichier : PREVIOUS_APPLICATION

In [33]:
prev_app.shape

(1670214, 37)

In [34]:
prev_app.columns.to_list()

['SK_ID_PREV',
 'SK_ID_CURR',
 'NAME_CONTRACT_TYPE',
 'AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'AMT_GOODS_PRICE',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'FLAG_LAST_APPL_PER_CONTRACT',
 'NFLAG_LAST_APPL_IN_DAY',
 'RATE_DOWN_PAYMENT',
 'RATE_INTEREST_PRIMARY',
 'RATE_INTEREST_PRIVILEGED',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'DAYS_DECISION',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'SELLERPLACE_AREA',
 'NAME_SELLER_INDUSTRY',
 'CNT_PAYMENT',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION',
 'NFLAG_INSURED_ON_APPROVAL']

In [35]:
# Traitement des variables temporelles
days_col = ['DAYS_FIRST_DRAWING',
            'DAYS_FIRST_DUE',
            'DAYS_LAST_DUE_1ST_VERSION',
            'DAYS_LAST_DUE',
            'DAYS_TERMINATION',]

prev_app[days_col].describe()

,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
count,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000
mean,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838
std,88916.115833,72444.869708,106857.034789,149647.415123,153303.516729
min,-2922.000000,-2892.000000,-2801.000000,-2889.000000,-2874.000000
25%,365243.000000,-1628.000000,-1242.000000,-1314.000000,-1270.000000
50%,365243.000000,-831.000000,-361.000000,-537.000000,-499.000000
75%,365243.000000,-411.000000,129.000000,-74.000000,-44.000000
max,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000


Tout comme sur le fichier application_train, des outliers apparaissent (365 243), notamment sur la colonne 'DAYS_FIRST_DRAWING'.

In [36]:
# Remplacement des outliers par NaN
days_outliers = prev_app['DAYS_FIRST_DRAWING'].max()
prev_app[days_col] = prev_app[days_col].replace({days_outliers : np.nan})

print('Remplacement de {} par NaN'.format(days_outliers))

Remplacement de 365243.0 outliers par NaN


In [37]:
# Création de feature : taux d'obtention de crédit
prev_app['APP_CREDIT_PERCENT'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']

In [38]:
# Variables catégorielles binaires : factorisation
binary_feat_prev = [col for col in prev_app.select_dtypes(object).columns if prev_app[col].nunique() == 2]
print('Nombre de variables catégorielles binaires PREVIOUS_APPLICATION : {}'.format(len(binary_feat_prev)))

for col in binary_feat_prev:
    prev_app[col], uniques = pd.factorize(prev_app[col])
display(prev_app[binary_feat_prev].head())

Nombre de variables catégorielles binaires PREVIOUS_APPLICATION : 1


,FLAG_LAST_APPL_PER_CONTRACT
0,0
1,0
2,0
3,0
4,0


In [39]:
# Variables catégorielles non binaires (OneHotEncoder)
prev_app, prev_encoded_cols = one_hot_encoding(prev_app, nan_as_category = True)
display(prev_app.head())
display(prev_app.shape)

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,PRODUCT_COMBINATION_nan
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,0,1,...,False,False,False,False,False,True,False,False,False,False
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,0,1,...,False,False,False,False,False,False,False,False,False,False
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,0,1,...,False,False,False,False,False,False,False,False,False,False
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,0,1,...,True,False,False,False,False,False,False,False,False,False
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,0,1,...,False,False,False,False,False,False,False,False,False,False


(1670214, 179)

In [40]:
# Définition des aggrégations
prev_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERCENT': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum']}

for col in prev_encoded_cols:
    prev_aggregations[col] = ['mean']

In [41]:
# Regroupement par SK_ID_CURR
prev_grouped = prev_app.groupby('SK_ID_CURR').agg(prev_aggregations)

# Renommage des colonnes
prev_grouped.columns = rename_grouped_index(prev_grouped, prefix = 'PREV_')

# Fusion avec df_global
df_global = df_global.join(prev_grouped, how = 'left', on = 'SK_ID_CURR')

print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 494)


#### Fichier : POS_CASH_BALANCE

In [42]:
pos_cash_bal.shape

(10001358, 8)

In [43]:
# Variables catégorielles non binaires (OneHotEncoder)
pos_cash_bal, pos_encoded_cols = one_hot_encoding(pos_cash_bal, nan_as_category = True)
display(pos_cash_bal.head())
display(pos_cash_bal.shape)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_nan
0,1803195,182943,-31,48.0,45.0,0,0,True,False,False,False,False,False,False,False,False,False
1,1715348,367990,-33,36.0,35.0,0,0,True,False,False,False,False,False,False,False,False,False
2,1784872,397406,-32,12.0,9.0,0,0,True,False,False,False,False,False,False,False,False,False
3,1903291,269225,-35,48.0,42.0,0,0,True,False,False,False,False,False,False,False,False,False
4,2341044,334279,-35,36.0,35.0,0,0,True,False,False,False,False,False,False,False,False,False


(10001358, 17)

In [44]:
# Définition des aggrégations
pos_cash_aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']}

for cat in pos_encoded_cols:
    pos_cash_aggregations[cat] = ['mean']
print(pos_cash_aggregations)

{'MONTHS_BALANCE': ['max', 'mean', 'size'], 'SK_DPD': ['max', 'mean'], 'SK_DPD_DEF': ['max', 'mean'], 'NAME_CONTRACT_STATUS_Active': ['mean'], 'NAME_CONTRACT_STATUS_Amortized debt': ['mean'], 'NAME_CONTRACT_STATUS_Approved': ['mean'], 'NAME_CONTRACT_STATUS_Canceled': ['mean'], 'NAME_CONTRACT_STATUS_Completed': ['mean'], 'NAME_CONTRACT_STATUS_Demand': ['mean'], 'NAME_CONTRACT_STATUS_Returned to the store': ['mean'], 'NAME_CONTRACT_STATUS_Signed': ['mean'], 'NAME_CONTRACT_STATUS_XNA': ['mean'], 'NAME_CONTRACT_STATUS_nan': ['mean']}


In [45]:
# Regroupement par SK_ID_CURR
pos_cash_grouped = pos_cash_bal.groupby('SK_ID_CURR').agg(pos_cash_aggregations)

# Renommage des colonnes
pos_cash_grouped.columns = rename_grouped_index(pos_cash_grouped, prefix = 'POS_')

display(pos_cash_grouped.head())

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN,POS_NAME_CONTRACT_STATUS_Approved_MEAN,POS_NAME_CONTRACT_STATUS_Canceled_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Demand_MEAN,POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_XNA_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,
100001,-53,-72.555556,9,7,0.777778,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0
100002,-1,-10.000000,19,0,0.000000,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
100003,-18,-43.785714,28,0,0.000000,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0
100004,-24,-25.500000,4,0,0.000000,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0
100005,-15,-20.000000,11,0,0.000000,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0


In [46]:
# Création de feature : 
pos_cash_grouped['POS_COUNT'] = pos_cash_bal.groupby('SK_ID_CURR').size()

In [47]:
# Fusion avec df_global
df_global = df_global.join(pos_cash_grouped, how = 'left', on = 'SK_ID_CURR')

print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 512)


#### Fichier : INSTALLMENTS_PAYMENTS

In [48]:
display(install_pay.head())
display(install_pay.shape)
display(install_pay.info())

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


(13605401, 8)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   SK_ID_CURR              int64  
 2   NUM_INSTALMENT_VERSION  float64
 3   NUM_INSTALMENT_NUMBER   int64  
 4   DAYS_INSTALMENT         float64
 5   DAYS_ENTRY_PAYMENT      float64
 6   AMT_INSTALMENT          float64
 7   AMT_PAYMENT             float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


None

In [49]:
# Création de features (sans valeurs négatives)

install_pay['PAYMENT_PERCENT'] = install_pay['AMT_PAYMENT'] / install_pay['AMT_INSTALMENT']
install_pay['PAYMENT_DIFF'] = install_pay['AMT_INSTALMENT'] - install_pay['AMT_PAYMENT']

install_pay['DAYS_PAST_DUE'] = install_pay['DAYS_ENTRY_PAYMENT'] - install_pay['DAYS_INSTALMENT']
install_pay['DAYS_PAST_DUE'] = install_pay['DAYS_PAST_DUE'].apply(lambda x: x if x > 0 else 0)

install_pay['DAYS_BEFORE_DUE'] = install_pay['DAYS_INSTALMENT'] - install_pay['DAYS_ENTRY_PAYMENT']
install_pay['DAYS_BEFORE_DUE'] = install_pay['DAYS_BEFORE_DUE'].apply(lambda x: x if x > 0 else 0)

In [50]:
# Définition des aggrégations
install_pay_aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DAYS_PAST_DUE': ['max', 'mean', 'sum'],
    'DAYS_BEFORE_DUE': ['max', 'mean', 'sum'],
    'PAYMENT_PERCENT': ['max', 'mean', 'sum', 'var'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}
print(install_pay_aggregations)

{'NUM_INSTALMENT_VERSION': ['nunique'], 'DAYS_PAST_DUE': ['max', 'mean', 'sum'], 'DAYS_BEFORE_DUE': ['max', 'mean', 'sum'], 'PAYMENT_PERCENT': ['max', 'mean', 'sum', 'var'], 'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'], 'AMT_INSTALMENT': ['max', 'mean', 'sum'], 'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'], 'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']}


In [51]:
# Regroupement par SK_ID_CURR
install_pay_grouped = install_pay.groupby('SK_ID_CURR').agg(install_pay_aggregations)

# Renommage des colonnes
install_pay_grouped.columns = rename_grouped_index(install_pay_grouped, prefix = 'INSTAL_')

install_pay_grouped.shape

(339587, 25)

In [52]:
# Création feature : nombre installments
install_pay_grouped['INSTAL_COUNT'] = install_pay.groupby('SK_ID_CURR').size()

# Fusion avec df_global
df_global = df_global.join(install_pay_grouped, how = 'left', on = 'SK_ID_CURR')

print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 538)


#### Fichier : CREDIT_CARD_BALANCE

In [53]:
cred_card_bal.shape

(3840312, 23)

In [54]:
# Variables catégorielles non binaires (OneHotEncoder)
cred_card_bal, cc_encoded_cols = one_hot_encoding(cred_card_bal, nan_as_category = True)
display(cred_card_bal.head())
display(cred_card_bal.shape)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_nan
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0,0,True,False,False,False,False,False,False,False
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,0,0,True,False,False,False,False,False,False,False
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,0,0,True,False,False,False,False,False,False,False
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,0,0,True,False,False,False,False,False,False,False
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,0,0,True,False,False,False,False,False,False,False


(3840312, 30)

In [55]:
# Suppression de 'SK_ID_PREV'
cred_card_bal = cred_card_bal.drop(columns='SK_ID_PREV')

# Aggrégation et regroupement par 'SK_ID_CURR'
cc_grouped = cred_card_bal.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])

# Renommage des colonnes
cc_grouped.columns = rename_grouped_index(cc_grouped, prefix = 'CC_')

display(cc_grouped.head())

,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_VAR,...,CC_NAME_CONTRACT_STATUS_Signed_MIN,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,3.5,0.0,0.00,0.000000,0.000,0.000000e+00,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100011,-75,-2,-38.5,-2849,462.5,0.0,189000.00,54482.111149,4031676.225,4.641321e+09,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100013,-96,-1,-48.5,-4656,776.0,0.0,161420.22,18159.919219,1743352.245,1.869473e+09,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100021,-18,-2,-10.0,-170,25.5,0.0,0.00,0.000000,0.000,0.000000e+00,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0
100023,-11,-4,-7.5,-60,6.0,0.0,0.00,0.000000,0.000,0.000000e+00,...,False,False,0.0,0,0.0,False,False,0.0,0,0.0


In [56]:
# Création de feature : nombre de lignes de crédit
cc_grouped['CC_COUNT'] = cred_card_bal.groupby('SK_ID_CURR').size()

# Fusion avec df_global
df_global = df_global.join(cc_grouped, how = 'left', on = 'SK_ID_CURR')

print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 679)


## 2. Etude des corrélations

In [57]:
# Matrice de corrélation
corr = df_global.drop(columns=['SK_ID_CURR', 'TARGET']).corr().abs()

corr_matrix = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
display(corr_matrix)

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
NAME_CONTRACT_TYPE,NaN,0.006553,0.004186,0.063077,0.028722,0.004817,0.200180,0.228259,0.166608,0.023615,...,0.001809,0.002210,0.003266,0.002950,NaN,NaN,NaN,NaN,NaN,0.009469
CODE_GENDER,NaN,NaN,0.343453,0.041581,0.048262,0.078535,0.016907,0.076729,0.018231,0.013552,...,0.003695,0.002490,0.006597,0.002373,NaN,NaN,NaN,NaN,NaN,0.000058
FLAG_OWN_CAR,NaN,NaN,NaN,0.002101,0.103053,0.089024,0.116985,0.143507,0.121231,0.039093,...,0.003263,0.002271,0.004520,0.002366,NaN,NaN,NaN,NaN,NaN,0.041355
FLAG_OWN_REALTY,NaN,NaN,NaN,NaN,0.001882,0.004256,0.034738,0.001626,0.042337,0.018158,...,0.005264,0.002326,0.005467,0.002753,NaN,NaN,NaN,NaN,NaN,0.050255
CNT_CHILDREN,NaN,NaN,NaN,NaN,NaN,0.014070,0.005996,0.025998,0.002254,0.024210,...,0.003177,0.005135,0.006419,0.007498,NaN,NaN,NaN,NaN,NaN,0.036949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CC_NAME_CONTRACT_STATUS_nan_MAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC_NAME_CONTRACT_STATUS_nan_MEAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC_NAME_CONTRACT_STATUS_nan_SUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CC_NAME_CONTRACT_STATUS_nan_VAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Variables avec corrélation supérieure au seuil (threshold)
threshold = 0.98
col_to_drop = [column for column in corr_matrix.columns if any(corr_matrix[column] > threshold)]

print('Nb de variables avec corrélation supérieure à {} : {}'.format(threshold, len(col_to_drop)))
print()
print(col_to_drop)

Nb de variables avec corrélation supérieure à 0.98 : 73

['AMT_GOODS_PRICE', 'YEARS_BUILD_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'OBS_60_CNT_SOCIAL_CIRCLE', 'NAME_INCOME_TYPE_Pensioner', 'ORGANIZATION_TYPE_XNA', 'BUREAU_MONTHS_BALANCE_MIN_MIN', 'BUREAU_CREDIT_ACTIVE_Closed_MEAN', 'PREV_AMT_CREDIT_MAX', 'PREV_AMT_GOODS_PRICE_MAX', 'PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN', 'PREV_NAME_PAYMENT_TYPE_XNA_MEAN', 'PREV_CODE_REJECT_REASON_CLIENT_MEAN', 'PREV_NAME_GOODS_CATEGORY_XNA_MEAN', 'PREV_PRODUCT_COMBINATION_nan_MEAN', 'POS_COUNT', 'INSTAL_PAYMENT_PERCENT_SUM', 'INSTAL_AMT_PAYMENT_MAX', 'INSTAL_AMT_PAYMENT_SUM', 'CC_MONTHS_BALANCE_MEAN', 'CC_MONTHS_BALANCE_SUM', 'CC_MONTHS_BALANCE_VAR', 'CC_AMT_PAY

In [59]:
# Suppression de ces colonnes 
df_global = df_global.drop(columns = col_to_drop)
print('Nouveau format : {}'.format(df_global.shape))

Nouveau format : (356249, 606)


## 3. Valeurs manquantes

In [60]:
# Comptage valeurs nulles
null_values = df_global.isna().sum().sum()
print("Valeurs nulles = {} ({:.2%})".format(null_values, null_values/df_global.size))

Valeurs nulles = 42464243 (19.67%)


## 4. Valeurs infinies

In [62]:
# Remplacement par NaN
df_global = df_global.replace([np.inf, -np.inf], np.nan)

## 5. Export des datasets

In [63]:
# Split selon liste IDs définie plus haut
df_test = df_global[df_global['SK_ID_CURR'].isin(test_ids)]
df_train = df_global[~df_global['SK_ID_CURR'].isin(test_ids)]

print('Format df_test : {}'.format(df_test.shape))
print('Format df_train : {}'.format(df_train.shape))

Format df_test : (48742, 606)
Format df_train : (307507, 606)


In [64]:
# Sauvegarde des datasets (.csv)
df_test.drop(columns = ['TARGET']).to_csv('./preprocessed_data/df_test.csv')
df_train.to_csv('./preprocessed_data/df_train.csv')

In [65]:
# Sauvegarde des datasets en versions compressées (.csv / gzip)
df_test.drop(columns = ['TARGET']).to_csv('./preprocessed_data/df_test_compressed.gzip', compression='gzip')
df_train.to_csv('./preprocessed_data/df_train_compressed.gzip', compression='gzip')